In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
# from tqdm import tqdm
import re
from random import randint
from time import sleep
import pickle
from tqdm.notebook import tqdm

In [5]:
addr = [
    "https://irecommend.ru/content/sait-skillbox-onlain-shkola",
    "https://otzovik.com/reviews/kursi_programmirovaniya_skillbox_russia_moscow/",
    "https://mooc.ru/company/skillbox/reviews",
    "https://yandex.ru/maps/org/skillbox/4275407173/reviews/?ll=37.607031%2C55.727789&z=14",
    "https://otzovik.com/reviews/netology_ru-onlayn_universitet_netologiya/",
    "https://irecommend.ru/content/sait-netologiya",
    "https://mooc.ru/company/netology/reviews",
    "https://yandex.ru/maps/org/netologiya/205031471256/reviews/?ll=37.624381%2C55.704101&z=14",
    "https://otzovik.com/reviews/geekbrains_ru-obuchayuschiy_portal_dlya_programmistov/",
    "https://mooc.ru/company/geekbrains/reviews",
    "https://irecommend.ru/content/sait-geekbrains",
    "https://yandex.ru/maps/org/geekbrains/1402263817/reviews/?ll=37.536922%2C55.796562&z=15",
    "https://irecommend.ru/content/sait-skillfactoryru",
    "https://otzovik.com/reviews/kurs_python_dlya_analiza_dannih-onlayn_shkola_skillfactory_russia_moscow/",
    "https://mooc.ru/company/skillfactory/reviews",
    "https://yandex.ru/maps/org/skillfactory/237135461560/reviews/?ll=37.622473%2C55.772409&z=14",
    "https://otzovik.com/reviews/yandeks_praktikum-servis_onlayn-obrazovaniya/",
    "https://mooc.ru/company/yandex/reviews",
    "https://irecommend.ru/content/kompyuternaya-programma-yandeks-praktikum"
]

In [6]:
irecommend = [i for i in addr if "irecommend" in i]
otzovik = [i for i in addr if "otzovik" in i]
mooc = [i for i in addr if "mooc.ru" in i]
yandex = [i for i in addr if "yandex.ru" in i]

# `OTZOVIK`

In [4]:
headers = {'Accept' : 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
'Accept-Encoding' : 'gzip, deflate, br',
'Accept-Language' : 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
'Connection' : 'keep-alive',
'Cache-Control' : 'no-cache',
'Cookie' : 'ssid=4098110196; refreg=1660531813~; ROBINBOBIN=38066c28028a9fcbab01227d72; guid=8f9aa153d6a514f63b1e55fdf028f9f9', 
# 'Referer' : ref, 
'Upgrade-Insecure-Requests' : '1',
'User-Agent' : 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:101.0) Gecko/20100101 Firefox/101.0'}

In [5]:
def get_review_data(url):
    '''Функция, собирающая данные с отзыва.'''
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.content, 'html.parser')

    user_name = soup.find('img', class_='photo')['alt']

    text = soup.find('div', class_='review-body description').text.replace('"', '""') # Для вывода в CSV меняем кавычки на двойные кавычки
    date = soup.find('abbr', class_='value')['title']
    rating = soup.find('abbr', class_='rating')['title']
    plus = soup.find('div', class_='review-plus').text.replace('"', '""')
    minus = soup.find('div', class_='review-minus').text.replace('"', '""')
    res ={}
    res['company'] = soup.find('a', class_="product-name")['title']
    res['plus'] = plus
    res['minus'] = minus
    res['text'] = text
    res['date'] = date
    res['rating'] = rating
    res['user_name'] = user_name
    res['url'] = url
    return res

In [6]:
def get_links(ref):

    req = requests.get(ref, headers=headers)
    soup = BeautifulSoup(req.content, 'html.parser')
    # print(soup)

    all_links = [] #Линки на отзывы

    #Сбор ссылок на все страницы
    while True:
        links = [i['href'] for i in soup.find_all('a', class_='review-title')] #Собираем ссылки на отзывы на этой странице
        all_links.extend(links)

        next_page = soup.find('a', class_='pager-item next tooltip-top') #Линк на следующую страницу с отзывами
        if next_page:
            req = requests.get('https://otzovik.com' + next_page['href'], headers=headers)
            soup = BeautifulSoup(req.content, 'html.parser')
        else:
            break
    print(f"Получено {len(all_links)} ссылок")
    assert len(all_links) != 0
    return all_links

In [7]:
def get_data(all_links, df):
    for n, link in enumerate(all_links):
        if "https://otzovik.com" + link not in df['url'].values:
            try:
                df = pd.concat((df, pd.DataFrame(get_review_data("https://otzovik.com" + link, company_id), index=[0])))
                print(f"Обработано {n} ссылок", end="\r")
            except Exception as e:
                print(e)
                break
        
    return df

In [59]:
res_df = pd.read_csv("../data/raw/online_schools/otzovik.csv", index_col=[0])
# res_df = pd.DataFrame()
# res_df['url'] = 0

In [9]:
res_df

,url,company,plus,minus,text,date,rating,user_name
0,https://otzovik.com/review_9011559.html,Все отзывы о Курсы программирования Skillbox (...,"Преподаватели, формат обучения, рассрочка",Цена,Я начинал учить Python с нуля самостоятельно. ...,2019-11-13,4,SerGeor
0,https://otzovik.com/review_8077932.html,Все отзывы о Курсы программирования Skillbox (...,"Удобный график обучения, формат обучения, каче...",Иногда задерживается проверка заданий,"Рисованием увлекаюсь с детства, но недавно под...",2019-04-19,4,MaryPovarkova
0,https://otzovik.com/review_6155102.html,Все отзывы о Курсы программирования Skillbox (...,учитывают ситуацию своих клиентов - готовы пой...,стоимость (без акции),Оказавшись в неприятной ситуации связанной с т...,2018-03-05,4,bostut
0,https://otzovik.com/review_8613392.html,Все отзывы о Курсы программирования Skillbox (...,"Качество подачи знаний, профессионализм курато...",Минусов не увидел,"Курс рассчитан на 12 месяцев, я сейчас прошел ...",2019-08-14,5,AgVasnichev
0,https://otzovik.com/review_9089509.html,Все отзывы о Курсы программирования Skillbox (...,"Хорошие учителя, достойные практики и фишки, о...","Некоторым курсы кажутся либо слишком легкими, ...",Решил изучать PHP с нуля до про в Скиллбокс из...,2019-11-29,5,Delon
...,...,...,...,...,...,...,...,...
0,https://otzovik.com/review_11745379.html,Все отзывы о Яндекс Практикум - сервис онлайн-...,"удобная платформа, бесплатный пробный блок, бл...","баги в тренажере, местами сжатое и непонятное ...","Прохожу сейчас курс Python-разработчик, ближе ...",2021-04-05,3,sergey1580
0,https://otzovik.com/review_11744151.html,Все отзывы о Яндекс Практикум - сервис онлайн-...,"много практики, можно самому строить свой граф...","есть темы, которым стоит уделить немного больш...","Из-за пандемии отрасль, в которой я работала п...",2021-04-05,4,Abeja
0,https://otzovik.com/review_11722772.html,Все отзывы о Яндекс Практикум - сервис онлайн-...,"Интересная и насыщенная программа, затягивающи...",Нет такого.,Изначально выбирала между Нетологией и Яндекс ...,2021-04-01,5,Lizaveta1988
0,https://otzovik.com/review_10716732.html,Все отзывы о Яндекс Практикум - сервис онлайн-...,Теплый коллектив,Стоит немного реорганизовать обратную связь,На данный момент я еще прохожу курс от Яндекс....,2020-10-02,4,daha832


In [10]:
for ref in otzovik:
    print(f"Обрабатываем ссылку {ref}")

    all_links = get_links(ref)
    for n, link in enumerate(all_links):
        if ("https://otzovik.com" + link) not in res_df['url'].values:
            try:
                res_df = pd.concat((res_df, pd.DataFrame(get_review_data("https://otzovik.com" + link), index=[0])))
                print(f"Обработано {n} ссылок", end="\r")
            except Exception as e:
                print(e)
                break
    print("Done!")

Обрабатываем ссылку https://otzovik.com/reviews/kursi_programmirovaniya_skillbox_russia_moscow/
Получено 0 ссылок


AssertionError: 

In [ ]:
res_df.to_csv("../data/raw/online_schools/otzovik.csv")

In [60]:
res_df.shape

(2069, 8)

# `irecommend`

In [15]:
headers_ = {
    "Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Encoding" : "gzip, deflate, br",
    "Accept-Language" : "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
    "Connection" : "keep-alive",
    # "Cookie" : "s_uid=16614820923153135; stats_u_a=lgejCkIr%2BgB9avMj9v1UDutit45MuksMvb7l8IKzrfJBoRlyTJLDS1fkEHoCwRfjpjn5OQ71evt2rlD%2FhKXXVDdKrOZB0%2BAoU%2BV232tE8hI%3D; ab_var=6; stats_s_a=s8dDbzogg0p9rPje0Yyp%2FQNfPaz7TlnBrxfjoSGsOJMJ7II%2FUeQYY9SHWOET6Vilgqjm6b7mjlfd6HSXG2IYYQh5kADJVG4aJjexWrOGIFEdBgeNpB9%2B%2BZfbHwogVwBF6LXBpZgL9Y03VexdG2jtKL3OZwTTsmc%2BqfnC6up%2BT9jcC6vVWrlbmVvA97A6c8rv; e=6413166; statsactivity=6; statstimer=804; v=7c",
    "Host" : "irecommend.ru",
    "User-Agent" : "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:101.0) Gecko/20100101 Firefox/101.0"
}

In [5]:
from fake_useragent import UserAgent

In [14]:
# ua = UserAgent()
# headers_ = {
#     'User-Agent': ua.chrome,
#     'accept': 'application/json, text/javascript, */*; q=0.01',
#     'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
#     'origin': 'https://irecommend.ru'
# }

In [13]:
def parse_links_irecom(page, num):
    
    if num > 0:
        ref_ = page + f"?page={num}"
    else:
        ref_ = page
    _req = requests.get(ref_, headers=headers_)
    _soup = BeautifulSoup(_req.content, 'html.parser')
    page_links = []
    for i in _soup.findAll('div', class_="reviewTitle"):
        try:
            page_links.append(i.find("a")['href'])
        except:
            pass
    print(f"Страница {num} всего {len(page_links)} ссылок")
    return page_links

In [12]:
def get_links_irecom(link):
    req = requests.get(link, headers=headers_)
    soup = BeautifulSoup(req.content, 'html.parser')
    _all_links = []
    try:
        number_pages = int(soup.find("li",  class_="pager-last").find("a")['title'].split()[-1])
    except:
        number_pages = 1
    for num_page in range(number_pages+1):
        _all_links.extend(parse_links_irecom(link, num_page))
    return set(_all_links)

In [61]:
df_irecom = pd.read_csv("../data/raw/online_schools/irecommend.csv", index_col=[0])
# df_irecom = pd.DataFrame()
# df_irecom['review_url'] = 0


In [11]:
def get_review_data_irecom(url, proxy):
    _r = requests.get(url, headers=headers_, proxies={'http': proxy, 'https': proxy}, timeout=1.5)
    _soup = BeautifulSoup(_r.text)


    review_text = _soup.find('div', {'itemprop': 'reviewBody'}).text

    review_title = _soup.select('h2.reviewTitle a')[0].text

    review_plus_list = _soup.select(
        '#content > div.review-node > div.reviewBlock > div.ratio > div.plus > ul')
    review_plus = "\n".join(review_plus_list[0].stripped_strings) if len(review_plus_list) else ''

    review_minus_list = _soup.select(
        '#content > div.review-node > div.reviewBlock > div.ratio > div.minus > ul')
    review_minus = "\n".join(review_minus_list[0].stripped_strings) if len(review_minus_list) else ''

    review_verdict_list = _soup.select(
        '#content > div.review-node > div.reviewBlock > div.conclusion > span.verdict')
    review_verdict = review_verdict_list[0].text if len(review_verdict_list) else ''

    review_usage_experience_html = _soup.find(text='Опыт использования:')
    review_usage_experience = review_usage_experience_html.findNext('div').contents[
        0] if review_usage_experience_html else ''

    review_cost_html = _soup.find(text='Стоимость:')
    review_cost = review_cost_html.findNext('div').contents[0] if review_cost_html else ''

    review_raiting = len(_soup.select(
        '#content > div.review-node > div.reviewBlock > div.reviewHeader > div.authorBlock > div.text > div.starsRating > div > div > div.on'))

    review_average_rating_list = _soup.select(
        'div.rating > div.description > a > div > div > div.description > div > span.average-rating > span')
    sleep(1)
    review_average_rating = review_average_rating_list[0].text if len(review_average_rating_list) else ''

    review_total_votes = _soup.find('span', {'class': 'total-votes'}).span.text

    review_category_html = _soup.find(text=' Категория:')
    review_category = review_category_html.findNext('a').contents[0] if review_category_html else ''

    review_product_type_html = _soup.find(text=' Тип товаров:')
    review_product_type = review_product_type_html.findNext('a').contents[0] if review_product_type_html else ''

    review_user = _soup.find('strong', {'class': 'reviewer'}).a.text

    review_user_url = _soup.find('a', {'title': 'Информация о пользователе.'}).get('href')
    review_user_url_full = 'https://irecommend.ru' + review_user_url
    # sleep(randint(2, 3))
    # r_user = requests.get(review_user_url_full, headers=headers_)
    # user_soup = BeautifulSoup(r_user.text)
    # review_user_reviews_count_string_list = user_soup.select(
    #     '#block-site-0 > div.content.text > div.breadcrumb > div > a')
    # review_user_reviews_count_string = review_user_reviews_count_string_list[0].text if len(
    #     review_user_reviews_count_string_list) else ''
    # extract_ints = re.findall(r'\d+', review_user_reviews_count_string)
    # review_user_reviews_count = extract_ints[0]

    result = {
        'review_url': url,
        'review_user': review_user,
        'review_user_url': review_user_url_full,
        # 'review_user_reviews_count': review_user_reviews_count,
        'review_text': review_text,
        'review_title': review_title,
        'review_plus': review_plus,
        'review_minus': review_minus,
        'review_verdict': review_verdict,
        'review_usage_experience': review_usage_experience,
        'review_cost': review_cost,
        'review_raiting': review_raiting,
        'review_average_rating': review_average_rating,
        'review_total_votes': review_total_votes,
        'review_category': review_category,
        'review_product_type': review_product_type
    }
    return result

In [30]:
_all_links = get_links_irecom(irecommend[4])
len(_all_links)

Страница 0 всего 50 ссылок
Страница 1 всего 13 ссылок
Страница 2 всего 13 ссылок


57

In [10]:
irecommend

['https://irecommend.ru/content/sait-skillbox-onlain-shkola',
 'https://irecommend.ru/content/sait-netologiya',
 'https://irecommend.ru/content/sait-geekbrains',
 'https://irecommend.ru/content/sait-skillfactoryru',
 'https://irecommend.ru/content/kompyuternaya-programma-yandeks-praktikum']

In [ ]:
skillbox_links = get_links_irecom(irecommend[0])
print(len(skillbox_links))
with open("../data/raw/online_schools/irecommend_skillbox_links.pickle", "wb") as f:
    pickle.dump(skillbox_links, f)

In [28]:
# netologya_links = get_links_irecom(irecommend[1])
len(netologya_links)
with open("../data/raw/online_schools/irecommend_netologya_links.pickle", "wb") as f:
    pickle.dump(netologya_links, f)

In [31]:
# _all_links = get_links_irecom(irecommend[2])
geekbrains_links = _all_links.copy()
with open("../data/raw/online_schools/irecommend_geekbrains_links.pickle", "wb") as f:
    pickle.dump(geekbrains_links, f)

In [21]:
# _all_links = get_links_irecom(irecommend[3])
skillfactory_links = _all_links.copy()
with open("../data/raw/online_schools/irecommend_skillfactory_links.pickle", "wb") as f:
    pickle.dump(skillfactory_links, f)

In [32]:
# _all_links = get_links_irecom(irecommend[4])
ya_praktikum_links = _all_links.copy()
with open("../data/raw/online_schools/irecommend_ya_praktikum_links.pickle", "wb") as f:
    pickle.dump(ya_praktikum_links, f)

In [33]:
from itertools import cycle
# proxies = pd.read_csv("../socks4.csv", header=None)[0].tolist()
proxies = pd.read_csv("../proxy.csv", header=None)[0].tolist()
proxy_pool = cycle(proxies)

# print(proxies)
proxy = next(proxy_pool)

In [ ]:
for i in range(100):
    try:
        
        for n, link in enumerate(_all_links):
            if ('https://irecommend.ru' + link) not in df_irecom['review_url'].values:
                proxy = next(proxy_pool)
                try:
                    df_irecom = pd.concat((df_irecom, pd.DataFrame(get_review_data_irecom('https://irecommend.ru' + link, proxy), index=[0])))
                    print(f"Обработано {n} ссылок", end="\r")
                except Exception as e:
                    print(e)
                    # break
    except:
        pass

In [36]:
df_irecom.to_csv("../data/raw/online_schools/irecommend.csv")

In [7]:
irecommend

['https://irecommend.ru/content/sait-skillbox-onlain-shkola',
 'https://irecommend.ru/content/sait-netologiya',
 'https://irecommend.ru/content/sait-geekbrains',
 'https://irecommend.ru/content/sait-skillfactoryru',
 'https://irecommend.ru/content/kompyuternaya-programma-yandeks-praktikum']

# `mooc`

In [7]:
mooc

['https://mooc.ru/company/skillbox/reviews',
 'https://mooc.ru/company/netology/reviews',
 'https://mooc.ru/company/geekbrains/reviews',
 'https://mooc.ru/company/skillfactory/reviews',
 'https://mooc.ru/company/yandex/reviews']

In [21]:
headers_ = {
    "Accept" : "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Encoding" : "gzip, deflate, br",
    "Accept-Language" : "ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3",
    "Connection" : "keep-alive",
    # "Cookie" : "s_uid=16614820923153135; stats_u_a=lgejCkIr%2BgB9avMj9v1UDutit45MuksMvb7l8IKzrfJBoRlyTJLDS1fkEHoCwRfjpjn5OQ71evt2rlD%2FhKXXVDdKrOZB0%2BAoU%2BV232tE8hI%3D; ab_var=6; stats_s_a=s8dDbzogg0p9rPje0Yyp%2FQNfPaz7TlnBrxfjoSGsOJMJ7II%2FUeQYY9SHWOET6Vilgqjm6b7mjlfd6HSXG2IYYQh5kADJVG4aJjexWrOGIFEdBgeNpB9%2B%2BZfbHwogVwBF6LXBpZgL9Y03VexdG2jtKL3OZwTTsmc%2BqfnC6up%2BT9jcC6vVWrlbmVvA97A6c8rv; e=6413166; statsactivity=6; statstimer=804; v=7c",
    # "Host" : "irecommend.ru",
    "User-Agent" : "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:101.0) Gecko/20100101 Firefox/101.0"
}

In [26]:
def get_links_mooc(main_link):
    _req = requests.get(main_link + "?page=50", headers=headers_)
    _soup = BeautifulSoup(_req.content, 'html.parser')
    num_pages = int(_soup.find("a", class_="link-btn link-prev")['href'].split("=")[-1])+1
    print("Всего страниц со ссылками ", num_pages)
    answers = []
    for i in range(1, num_pages):
        print(f"Обрабатываем страницу {i}", end="\r")
        req = requests.get(main_link + f"?page={i}", headers=headers_)
        soup = BeautifulSoup(req.content, 'html.parser')
        # print(soup)
        text = str(soup.select("script", {"review": "reviewBody"})[3]).split('"@type": "Review"')
        # print(text)
        for t in text[1:]:
            text_item = t
            text_item = text_item.replace("\n", "")
            text_item = text_item.replace("\r", "")
            text_item = text_item.replace('"', "")
            text_item = text_item.replace('}', "")
            text_item = text_item.replace('{', "")
            text_item = text_item.replace('        ,', "")
            text_item = text_item.replace('         ]        </script>', "")
            # print(text_item.split('reviewBody:')[1].strip())
            answers.append(text_item.split('reviewBody:')[1].strip())
        sleep(10)
    return answers
        

In [27]:
skillbox = get_links_mooc(mooc[0])

Всего страниц со ссылками  27


In [30]:
skillbox = pd.DataFrame(data=skillbox, columns=['review'])


In [32]:
skillbox['company'] = 'skillbox'
skillbox.head()

,review,company
0,Хочу поделиться своим опытом обучения в Skillb...,skillbox
1,Я бесконечно благодарна создателям платформы з...,skillbox
2,Я довольно долго интересовалась сферой дизайна...,skillbox
3,Год назад сменила работу и пришла в сферу марк...,skillbox
4,Хочу оставить отзыв о прохождении в Skillbox к...,skillbox


In [31]:
netology = get_links_mooc(mooc[1])

Всего страниц со ссылками  10


In [33]:
netology = pd.DataFrame(data=netology, columns=['review'])
netology['company'] = 'netology'
netology.head()

,review,company
0,"Прошёл курс «Бизнес-аналитик» с 0, т.к до этог...",netology
1,"Прошёл курс «Бизнес-аналитик» с 0, т.к до этог...",netology
2,Я прошла курс Ландшафтного дизайна в нетологии...,netology
3,Всем привет!Хочу поделиться рассказом о своём ...,netology
4,Изучаю иллюстрацию и рисование с нуля. Очень у...,netology


In [34]:
geekbrains = get_links_mooc(mooc[2])

Всего страниц со ссылками  33


In [35]:
geekbrains = pd.DataFrame(data=geekbrains, columns=['review'])
geekbrains['company'] = 'geekbrains'
geekbrains.head()

,review,company
0,Моя дочь 9 лет занималась 4 месяца на онлайн-к...,geekbrains
1,Знакомство с прфессией состоялось,geekbrains
2,"Мне очень нравится форма подачи, обучение дает...",geekbrains
3,Современный и интересный формат образования.,geekbrains
4,"Всё зависит от самого человека. Конечно, снача...",geekbrains


In [36]:
skillfactory = get_links_mooc(mooc[3])

Всего страниц со ссылками  9


In [37]:
skillfactory = pd.DataFrame(data=skillfactory, columns=['review'])
skillfactory['company'] = 'skillfactory'
skillfactory.head()

,review,company
0,"Все неплохо организовано, На платформе сразу в...",skillfactory
1,Всем привет!С июня месяца начал заниматься на ...,skillfactory
2,Я главный инженер производственного предприяти...,skillfactory
3,"Выбрала курсы дата аналитики, потому что хочу ...",skillfactory
4,Skillfactory - это действительно отличный прод...,skillfactory


In [38]:
yandex = get_links_mooc(mooc[4])

Всего страниц со ссылками  13


In [39]:
yandex = pd.DataFrame(data=yandex, columns=['review'])
yandex['company'] = 'yandex'
yandex.head()

,review,company
0,"В первую очередь хочется сообщить, что лично у...",yandex
1,Привет всем!Проходил курс от Яндекс практикума...,yandex
2,Я закончила курс по тестированию.Через 2 месяц...,yandex
3,"Я закончила курс интернет-маркетолога, который...",yandex
4,"ДОСТОИНСТВА: формат, практические задания, удо...",yandex


In [41]:
mooc_df = pd.concat((skillbox, netology))
mooc_df = pd.concat((mooc_df, geekbrains))
mooc_df = pd.concat((mooc_df, skillfactory))
mooc_df = pd.concat((mooc_df, yandex))


In [45]:
mooc_df = mooc_df.drop_duplicates()

In [46]:
mooc_df.to_csv("../data/raw/online_schools/mooc.csv")

# `yandex`

In [51]:
yandex

['https://yandex.ru/maps/org/skillbox/4275407173/reviews/?ll=37.607031%2C55.727789&z=14',
 'https://yandex.ru/maps/org/netologiya/205031471256/reviews/?ll=37.624381%2C55.704101&z=14',
 'https://yandex.ru/maps/org/geekbrains/1402263817/reviews/?ll=37.536922%2C55.796562&z=15',
 'https://yandex.ru/maps/org/skillfactory/237135461560/reviews/?ll=37.622473%2C55.772409&z=14']

In [1]:
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait

from bs4 import BeautifulSoup as bs

In [3]:
# driver = webdriver.Firefox()

# `ALL`

In [65]:
otzivik_df = pd.read_csv("../data/raw/online_schools/otzovik.csv", index_col=[0])
irecom_df = pd.read_csv("../data/raw/online_schools/irecommend.csv", index_col=[0])
mooc_df = pd.read_csv("../data/raw/online_schools/mooc.csv", index_col=[0])

In [66]:
with open("../data/raw/online_schools/irecommend_skillbox_links.pickle", "rb") as f:
    skillbox_links = pickle.load(f)
with open("../data/raw/online_schools/irecommend_netologya_links.pickle", "rb") as f:
    netologya_links = pickle.load(f)
with open("../data/raw/online_schools/irecommend_geekbrains_links.pickle", "rb") as f:
    geekbrains_links = pickle.load(f)
with open("../data/raw/online_schools/irecommend_skillfactory_links.pickle", "rb") as f:
    skillfactory_links = pickle.load(f)
with open("../data/raw/online_schools/irecommend_ya_praktikum_links.pickle", "rb") as f:
    ya_praktikum_links = pickle.load(f)


In [67]:
skillbox_links = ["https://irecommend.ru" + i for i in skillbox_links]
netologya_links = ["https://irecommend.ru" + i for i in netologya_links]
geekbrains_links = ["https://irecommend.ru" + i for i in geekbrains_links]
skillfactory_links = ["https://irecommend.ru" + i for i in skillfactory_links]
ya_praktikum_links = ["https://irecommend.ru" + i for i in ya_praktikum_links]

In [68]:
def get_company_name_irecom(skillbox_links, netologya_links, geekbrains_links, skillfactory_links, ya_praktikum_links, link):
    if link in skillbox_links:
        return "skillbox"
    elif link in netologya_links:
        return "netology"
    elif link in geekbrains_links:
        return "geekbrains"
    elif link in skillfactory_links:
        return "skillfactory"
    elif link in ya_praktikum_links:
        return "yandex"

In [69]:
irecom_df['company'] = irecom_df['review_url'].apply(lambda x: get_company_name_irecom(skillbox_links, netologya_links, geekbrains_links, skillfactory_links, ya_praktikum_links, x))

In [70]:
otzivik_df['company_name'] = otzivik_df['company']

In [71]:
def get_company_name_otzovik(name):
    if 'Skillbox' in name:
        return "skillbox"
    elif 'Нетология' in name:
        return 'netology'
    elif "Geekbrains" in name:
        return "geekbrains"
    elif "SkillFactory" in name:
        return "skillfactory"
    elif "Яндекс" in name:
        return "yandex"

In [72]:
otzivik_df['company'] = otzivik_df['company_name'].apply(get_company_name_otzovik)

In [73]:
otzivik_df['full_text'] = otzivik_df['plus'] + ' ' + otzivik_df['minus'] + ' ' + otzivik_df['text']

In [74]:
irecom_df['full_text'] = irecom_df['review_plus'] + ' ' + irecom_df['review_minus'] + ' ' + irecom_df['review_text']

In [89]:
from stop_words import get_stop_words
from string import punctuation
from pymorphy2 import MorphAnalyzer
import jamspell
from tqdm import tqdm
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
import collections

corrector = jamspell.TSpellCorrector()
corrector.LoadLangModel('../ru_small.bin')
tqdm.pandas()

In [76]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

In [77]:
def preprocess_text(txt):
    txt = str(txt)
    txt = corrector.FixFragment(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub('[^а-яА-Я]', ' ', txt)
    txt = re.sub("/sне ", "не", txt)
    txt = re.sub("за м", "займ", txt)
    txt = re.sub("заем", "займ", txt)
    txt = re.sub(" вс  ", " всё", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    # txt = re.sub(' заесть ', ' займ ', txt)
    return " ".join(txt)

In [80]:
otzivik_df[['full_text', 'company']].head(1)

,full_text,company
0,"Преподаватели, формат обучения, рассрочка Цена...",skillbox


In [81]:
irecom_df[['full_text', 'company']].head(1)

,full_text,company
0,Есть бесплатные вводные занятия\nЕсть ознакоми...,skillbox


In [86]:
mooc_df.rename(columns={'review' : 'full_text'}, inplace=True)

In [87]:
result_df = pd.concat((otzivik_df[['full_text', 'company']], irecom_df[['full_text', 'company']]))
result_df = pd.concat((result_df, mooc_df))
result_df

,full_text,company
0,"Преподаватели, формат обучения, рассрочка Цена...",skillbox
0,"Удобный график обучения, формат обучения, каче...",skillbox
0,учитывают ситуацию своих клиентов - готовы пой...,skillbox
0,"Качество подачи знаний, профессионализм курато...",skillbox
0,"Хорошие учителя, достойные практики и фишки, о...",skillbox
...,...,...
355,Аналитика данных — очень интересная специально...,yandex
356,"Достоинства: хороший тренажер, много знаний по...",yandex
357,Открыла для себя очень много нового в IT сфере...,yandex
358,Практикум дает реальную возможность добиться в...,yandex


In [88]:
result_df['text'] = result_df['full_text'].progress_apply(preprocess_text)

100%|██████████| 5068/5068 [04:26<00:00, 19.01it/s]


In [98]:
result_df.drop_duplicates(subset=['text'], inplace=True)

In [99]:
result_df.shape

(4891, 3)

In [100]:
def get_ngrams(text, n):
    n_grams = ngrams(word_tokenize(" ".join(text)), n)
    return [ ' '.join(grams) for grams in n_grams]

In [101]:
def get_ngrams_full(dataframe, column, company, num_grams, n_most_common=10):
    texts = ''
    words = []
    dataframe = dataframe[dataframe['company'] == company]
    for line in dataframe[column]: 
        line_words = line.split()
        lw = [w.lower() for w in line_words if w not in sw]
        words.extend(lw)
    
    c = collections.Counter()
    for word in get_ngrams(words, num_grams):
        c[word] += 1

    return c.most_common(n_most_common)

In [102]:
r_df = pd.DataFrame()
for company in tqdm(result_df['company'].unique()):
    res_dict = {}
    res_dict['Company'] = company
    res_dict['number_of_reviews'] = result_df[result_df['company'] == company].shape[0]
    res_dict['unigram'] = get_ngrams_full(result_df, 'text', company, 1)
    res_dict['bigram'] = get_ngrams_full(result_df, 'text', company, 2)
    res_dict['trigram'] = get_ngrams_full(result_df, 'text', company, 3)
    res_dict['4_igram'] = get_ngrams_full(result_df, 'text', company, 4)
    r_df = pd.concat((r_df, pd.DataFrame.from_dict(res_dict)))

100%|██████████| 5/5 [00:09<00:00,  1.98s/it]


In [103]:
r_df.to_csv("../data/processed/online_school.csv")